In [ ]:
import iris

data = iris.load("./data/precip.nc")

In [ ]:
print(data[0])

In [ ]:
cdata = list(data[0].data.flatten())
rgba = []
for c in cdata:
    r = int(c*255)
    b, g, a = 0, 0, 255
    rgba += [r, g, b, a]

In [ ]:
import json
from IPython.display import Javascript

javascript = 'window.data={};'.format(json.dumps(rgba))
Javascript(javascript)


create html element to render to

In [ ]:
%%html
<script type="x-shader/x-vertex" id="my_vert">
uniform vec2 offset;
varying vec2 vUv;

void main(){

  vUv = uv;

  // apply longitude offset
  vUv.x = vUv.x + offset.x;
  if (vUv.x > 1.0) {
    vUv.x = vUv.x - 1.0;
  }

  gl_Position = projectionMatrix * modelViewMatrix * vec4( position, 1.0 );
}
</script>
<script type="x-shader/x-fragment" id="my_frag">
uniform sampler2D dataTexture;
uniform vec2 dataShape;
uniform sampler2D colorMap;
varying vec2 vUv;

float remapValueToRange(float val, float low1, float high1, float low2, float high2) {
  if (val < 0.1) {
    return 0.0;
  }
  return low2 + (val - low1) * (high2 - low2) / (high1 - low1);
}

void main() {

  float dataValue = texture2D(dataTexture, vUv).r;
  vec4 colorLookup = texture2D(colorMap, vec2(dataValue, 0.5));
  vec4 color = vec4(colorLookup.r, colorLookup.g, colorLookup.b, remapValueToRange(dataValue, 0.0, 1.0, 0.3, 0.8));
  gl_FragColor = color;

}
</script>

In [ ]:
%%javascript
element.append("<div id='content'></div>");

Import js dependencies and boilerplate three.js code

In [ ]:
%%javascript
requirejs.config({
    paths: { 
        'T' : ['//cdnjs.cloudflare.com/ajax/libs/three.js/r83/three']
    },
});
require(['T'], function(THREE) {
    console.log(THREE);
    var parentDiv = document.getElementById("content");
    var camera, scene, ambientLight, directionalLight, renderer;
    var textureLoader, fileLoader;
    var width = 700;
    var height = 400;
    
    cleanUp();
    init();
    animate();
    
    function init(){
        textureLoader = new THREE.TextureLoader();
        fileLoader = new THREE.FileLoader();
        
        scene = new THREE.Scene();
        
        camera = new THREE.PerspectiveCamera( 75, width / height, 1, 1000 );
        camera.position.z = 2;
        
        ambientLight = new THREE.AmbientLight(0xffffff, 0.2); // soft white light
        scene.add(ambientLight);
        
        directionalLight = new THREE.DirectionalLight(0xbbbbbb, 0.8);
        directionalLight.position.copy(camera.position);
        scene.add(directionalLight);
        
        renderer = new THREE.WebGLRenderer({antialias: true});
        renderer.setSize(width, height);
        renderer.setClearColor('white');
        parentDiv.appendChild( renderer.domElement ) ;
        
        addGlobeMesh();
        addDataMesh();
    }  
    
    
    function animate(time){
        window.requestAnimationFrame( animate ); 
        renderer.render( scene, camera);
    }
    
    
    function addGlobeMesh() {
        var sat = textureLoader.load('./textures/sat.jpg', texture => texture);
        var bump = textureLoader.load('./textures/bump.jpg', texture => texture);
        var specular = textureLoader.load('./textures/specular.png', texture => texture);

        var geometry = new THREE.SphereGeometry(1, 32, 32);
        var material = new THREE.MeshPhongMaterial({
          map: sat,
          bumpMap: bump,
          bumpScale: 0.015,
          specularMap: specular,
          specular: new THREE.Color('grey'),
          shininess: 10
        });
        var mesh = new THREE.Mesh(geometry, material);

        scene.add(mesh);
    }
    
    
    function getPyDataTexture() {
        var raw = Uint8Array.from(window.data);
        var texture = new THREE.DataTexture(raw,800,600,THREE.RGBAFormat);
        texture.magFilter = THREE.LinearFilter;
        texture.minFilter = THREE.LinearFilter;
        texture.needsUpdate = true;
        return texture;
    }
    
    function addDataMesh() {
        var colorMap = textureLoader.load('./textures/viridis.png', texture => texture);
        
        var geometry = new THREE.SphereGeometry(1.01, 32, 32);
        var material = new THREE.ShaderMaterial({
          uniforms: {
            offset: {
              type: 'v2',
              value: new THREE.Vector2(0.5, 0.0)
            },
            dataTexture: {
              type: 't',
              value: getPyDataTexture()
            },
            dataShape: {
              type: 'v2',
              value: new THREE.Vector2(800, 600)
            },
            colorMap: {
              type: 't',
              value: colorMap
            }
          },
          vertexShader: $('#my_vert').text(),
          fragmentShader: $('#my_frag').text(),
          transparent: true   // required if you want to alter alpha levels in shader
        });
        var mesh = new THREE.Mesh(geometry, material);
        scene.add(mesh);
    }
    
    
    function cleanUp() {
        while (parentDiv.firstChild) {
            parentDiv.removeChild(parentDiv.firstChild);
        }
    }
    
});